# Signatures by sample heatmap/dot plot
- make dotplot forFigure S1
- signatures are from the paper table S2, grouped to 3

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import seaborn as sn

In [ ]:
import sys; sys.path.append("../resources/")
import dotplot_utils as cody_dot # copied from cody's resources/dotplot_utils.py into my directory

In [ ]:
import anndata as ad
sys.path.append("../resources/scRNA/")
import zc_function as zc

In [ ]:
# make output directory
import os
if not os.path.exists("scRNA_out"):
    os.mkdir("scRNA_out")

if not os.path.exists("scRNA_out/FFPE_scRNA/"):
    os.mkdir("scRNA_out/FFPE_scRNA/")

## load data and signature

In [ ]:
dat_path = '/home/lucy/Dropbox (VU Basic Sciences)/immune_exclusion/h5ad_files/'
dat = sc.read( dat_path + "outer_combined_all4_dat.h5ad")

In [ ]:
dat.obs.SampleId.unique()

In [ ]:
sigs = pd.read_csv("./tableS2.csv", header = 0, index_col = None) # suppl. table 2 from the paper
sigs.head()

In [ ]:
test_group = ['Cytotoxicity', 'Exhaustion']

In [ ]:
#sigs.iloc[:, 8:15 ].head()

## define dictionaries for plotting

In [ ]:
# copied from Cody's step3/scRNA_summaries.ipynb

cmap_dict = {
    # Tumor Type
    'SSL/HP':"#c4a4e1",'MSI-H':"#7a4fa3",'MSS':"#ffc101",'TA/TVA':"#fee799",'NL':"#1f77b4",
    # Tumor Location
    "Cecum":"#62a32e","Ascending":"#b0dc7d","Hepatic Flexure":"#ebf6db","Transverse":"#fbe6f1","Descending":"#eeadd4","Sigmoid":"#cf4191",
    # this one's global
    "nan":"#ffffff",
    # These are black and white for T and F
    "T":"#000000","F":"#ffffff",
}

In [ ]:
patient_id_ls = list(dat.obs.SampleId.unique()) 
print( list(dat.obs.SampleId.unique()) )

In [ ]:
tumor_type_dict = {'9142_s2':'MSS', '9142_s1':'MSI-H', 
                   '10096_s1':'MSI-H', '10096_s2':'MSS', '10096_s3':'MSS', '10096_s4':'MSI-H', 
                   '10180_01_s1': 'MSI-H', '10180_01_s2':'MSI-H', '10180_01_s3':'MSI-H', '10180_01_s4':'MSS', 
                   '10180_02_s1':'MSS', '10180_02_s2':'TA/TVA', '10180_02_s3':'MSS', '10180_02_s4':'MSS', 
                   '10284_s1':'MSI-H', '10284_s2':'MSS', '10284_s3':'MSS', '10284_s4':'MSI-H'}

In [ ]:
# mapping sample ID to tumor type cmap
patient_color_dict = dict(zip( patient_id_ls , [cmap_dict[tumor_type_dict[s] ] for s in patient_id_ls]) )

## test dotplot with 2 signatures
- pass test

In [ ]:
#dat.X.sum(axis = 1) #check if raw data

In [ ]:
dat.raw = dat.copy()

In [ ]:
zc.normalization(dat)

In [ ]:
for i in range(len( test_group)): # score a subset of the signatures
    gl = sigs[test_group[i]].dropna()
    sc.tl.score_genes(dat, gene_list=gl, score_name= test_group[i], use_raw = False)

In [ ]:
dat

In [ ]:
#sc.pl.umap(dat, color = 'Cytotoxicity')

In [ ]:
patient_color_dict

In [ ]:
tumor_type = [tumor_type_dict[s] for s in dat.obs.SampleId]

In [ ]:
dat.obs["Tumor_Type"] = tumor_type

In [ ]:
cody_dot.cody_heatmap(
        dat,
        groupby='SampleId',
        features=test_group,
        cluster_vars=False,
        vars_dict=None,
        groupby_order=None,
        groupby_colordict=patient_color_dict,
        
        
        save=None,
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )
#save="ST_out/plots_overview/{}_archetypes_dotplot.png".format(group_name)
#figsize=(width, archetypes_height)
#cluster_obs=True if groupby_order is None else False

## score all signatures

In [ ]:
dat.raw= dat.copy()

In [ ]:
zc.normalization(dat)

In [ ]:
sigs.columns

separate signatures into 3 groups : g1, g2, g3

In [ ]:
g1 = ['Cytotoxicity', 'Exhaustion', 'Cytokines', 'Chemokines', 'MHC', 'Fetal',
       'Stem', 'Metaplasia', 'Bacterial response', 'iCMS2_Up', 
       'iCMS3_Up', 'Proliferation', 'Translation']

In [ ]:
len(g1)

In [ ]:
g2 = [ 'T cell exhaustion', 'B cell', 'NK cell', 'Plasma cell',
       'T reg resting', 'T reg suppressive', 'T reg tissue homing',
       'T reg cytokines', 'T reg activation', 'TH1', 'TH2', 'TH17', 'TRM',
       'M1', 'M2', 'Costimulatory MYE', 'Stimulatory DCs',
       'IFN stimulated MYE', 'EMT', 'Senescence', 'SASP', 'IFN stimulated EPI',
       'Stress response', 'Fibrosis', 'T cell', 'Myeloid', 'Stroma', 'T reg',
       'T cell CD4', 'T cell CD8', 'cDC2', 'cDC1', 'Macrophage','Classical monocytes']

In [ ]:
len(g2)

In [ ]:
g3 = ['Cycle', 'Stress', 'Interferon', 'Hypoxia',
       'Metal', 'Mesenchymal', 'pEMT', 'Alveolar', 'Basal', 'Squamous',
       'Glandular', 'Ciliated', 'AC', 'OPC', 'NPC'] 

In [ ]:
len(g3)

In [ ]:
fond_dict = dict() # dictionary of signatures names as keys and genes found in the dataset's gene space as values

In [ ]:

for i in range(len(g1)):
    c = g1[i] # column name
    gl = sigs[c].dropna() #gene list
    gl2 = gl[np.isin(gl, dat.var_names)] # make sure all genes are in the dataset's gene space
    
    print(f"scoring {c}")
    print(f"{len(gl2)}/{len(gl)} genes scored for this signature")
    fond_dict[c] = gl2 # save the genes in the dound dictionary
    
    sc.tl.score_genes(dat, gene_list=gl2, score_name=c, use_raw=False) 
    

group 2 signatures

In [ ]:
for i in range(15):
    c = g2[i] # column name
    gl = sigs[c].dropna() #gene list
    gl2 = gl[np.isin(gl, dat.var_names)]
    
    print(f"scoring {c}")
    print(f"{len(gl2)}/{len(gl)} genes scored for this signature")
    fond_dict[c] = gl2
    
    sc.tl.score_genes(dat, gene_list=gl2, score_name=c, use_raw=False) 

In [ ]:
dat

In [ ]:
for i in range(15, len(g2)):
    c = g2[i] # column name
    gl = sigs[c].dropna() #gene list
    gl2 = gl[np.isin(gl, dat.var_names)]
    
    print(f"({i}) scoring {c}")
    print(f"{len(gl2)}/{len(gl)} genes scored for this signature")
    fond_dict[c] = gl2
    
    sc.tl.score_genes(dat, gene_list=gl2, score_name=c, use_raw=False) 

In [ ]:
for i in range(len(g3)):
    c = g3[i] # column name
    gl = sigs[c].dropna() #gene list
    gl2 = gl[np.isin(gl, dat.var_names)]
    
    print(f"({i}) scoring {c}")
    print(f"{len(gl2)}/{len(gl)} genes scored for this signature")
    fond_dict[c] = gl2
    
    sc.tl.score_genes(dat, gene_list=gl2, score_name=c, use_raw=False) 

## encode sample ID with patient ID names and combine replicates of each sample

In [ ]:
FFPE_tabel = pd.read_excel("./FFPE_metatable.xlsx", sheet_name='Sheet1', engine='openpyxl' , header = 0)

In [ ]:
FFPE_tabel

In [ ]:
#find replicate samples

In [ ]:
FFPE_tabel[FFPE_tabel.duplicated(['Block ID'])]

In [ ]:
FFPE_tabel[FFPE_tabel.duplicated(['Patient_label'])]

there are only two patient ID has duplicates, so find the duplicates' block ID and create a new patient ID col with suffix for different blocks

In [ ]:
new_patID = FFPE_tabel['Patient_label'].copy()
new_patID.head()

In [ ]:
FFPE_tabel[FFPE_tabel['Patient_label'] == 'PAT71397'] # they are from different blocks and should map to different plot IDs 

In [ ]:
new_patID[9] = 'PAT71397_1'
new_patID[10] = 'PAT71397_2'

In [ ]:
FFPE_tabel[FFPE_tabel['Patient_label'] == 'SG00001'] # they are replicates, should not change

In [ ]:
FFPE_tabel['plot_id'] = new_patID

## make dot plot

In [ ]:
tumor_type = [tumor_type_dict[s] for s in dat.obs.SampleId]

In [ ]:
dat.obs["Tumor_Type"] = tumor_type

### map patient ID to tumor type colors

In [ ]:
FFPE_tabel2 = FFPE_tabel[['plot_id', 'MSS status'] ] #a FFPE meta table with only unique plot_ids 
FFPE_tabel2.drop_duplicates()

make a cmap to map plot ID to MSS status color

In [ ]:
patient_color_dict2 = dict(zip( list(FFPE_tabel2['plot_id']) , [ cmap_dict[i ] for i in list( FFPE_tabel2['MSS status'] ) ] ) )

In [ ]:
patient_color_dict2

add plot id column to dat.obs

In [ ]:
sample_plot_id_dict = dict(zip(list(FFPE_tabel["SampleId"]), list(FFPE_tabel["plot_id"]) ))

In [ ]:
dat_plotId =[sample_plot_id_dict[s] for s in dat.obs.SampleId]

In [ ]:
dat.obs['plot_id'] = dat_plotId

sort plot id to match Cody's orders

In [ ]:
FFPE_tabel.sort_values(by=['MSS status', 'plot_id'], inplace=True)

In [ ]:
FFPE_tabel[['plot_id','MSS status'] ]

In [ ]:
print(list( FFPE_tabel['plot_id']) )

In [ ]:
ordered_plot_id = ['PAT71397_1', 'PAT30884', 'PAT59600', 'PAT59667', 'PAT71397_2', 'PAT71662', 'PAT73899', 'PAT74143', 'SG00003', 'SG00004',
                   'PAT00222', 'PAT33430', 'PAT54273', 'PAT59460', 'PAT73458', 'SG00001', 'SG00001', 'SG00002']

### plot

In [ ]:
cody_dot.cody_heatmap(
        dat,
        groupby='plot_id',
        features=g1,
        cluster_vars=False,
        vars_dict=None,
        groupby_order=ordered_plot_id,
        groupby_colordict=patient_color_dict2,
        
        figsize = [8,5],
        save=None,
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )
#save="ST_out/plots_overview/{}_archetypes_dotplot.png".format(group_name)
#figsize=(width, archetypes_height)
#cluster_obs=True if groupby_order is None else False

In [ ]:
cody_dot.cody_heatmap(
        dat,
        groupby='plot_id',
        features=g2,
        cluster_vars=False,
        vars_dict=None,
        groupby_order=ordered_plot_id,
        groupby_colordict=patient_color_dict2,
        
        figsize = [8,15],
        save=None,
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

In [ ]:
cody_dot.cody_heatmap(
        dat,
        groupby='plot_id',
        features=g3,
        cluster_vars=False,
        vars_dict=None,
        groupby_order=ordered_plot_id,
        groupby_colordict=patient_color_dict2,
        
        figsize = [8,5],
        save=None,
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

In [ ]:
sig_and_id = ['Cytotoxicity', 'Exhaustion', 'Cytokines', 'Chemokines', 'MHC', 'Fetal', 'Stem', 'Metaplasia', 'Bacterial response', 'iCMS2_Up', 'iCMS3_Up', 'Proliferation', 'Translation', 'Tumor_Type', 'T cell exhaustion', 'B cell', 'NK cell', 'Plasma cell', 'T reg resting', 'T reg suppressive', 'T reg tissue homing', 'T reg cytokines', 'T reg activation', 'TH1', 'TH2', 'TH17', 'TRM', 'M1', 'M2', 'Costimulatory MYE', 'Stimulatory DCs', 'IFN stimulated MYE', 'EMT', 'Senescence', 'SASP', 'IFN stimulated EPI', 'Stress response', 'Fibrosis', 'T cell', 'Myeloid', 'Stroma', 'T reg', 'T cell CD4', 'T cell CD8', 'cDC2', 'cDC1', 'Macrophage', 'Classical monocytes', 'Cycle', 'Stress', 'Interferon', 'Hypoxia', 'Metal', 'Mesenchymal', 'pEMT', 'Alveolar', 'Basal', 'Squamous', 'Glandular', 'Ciliated', 'AC', 'OPC', 'NPC', 'plot_id']

In [ ]:
dot_plot_signature_df = dat.obs[sig_and_id].copy()

In [ ]:
dot_plot_signature_df.head()

## save data

In [ ]:
dot_plot_signature_df.to_csv("scRNA_out/FFPE_scRNA/dotplot_signature_score_df.csv", header = True, index = True)